In [1]:
import gym
import numpy as np
import tensorflow as tf
env =  gym.make('FrozenLake8x8-v0')# initialize environment
#env =  gym.make('FrozenLake-v0')# initialize environment


[2016-07-04 18:01:42,405] Making new env: FrozenLake8x8-v0


In [2]:
class tf_RL_model:
    def __init__(self):
        self.action_table = {
            'L' : 0,
            'D' : 1,
            'R' : 2,
            'U' : 3
        }
        self.inv_action_table = {v: k for k, v in self.action_table.items()}
        
        # Learning paramters
        self.learning_rate = 0.1
        self.r_prob = 0.5
        self.r_prob_decay = 0.98

    def epsilon_greedy_action(self,state, ep):
        if np.random.rand() < self.r_prob:
            action = env.action_space.sample()
        else:
            action = sess.run(optimal_action, feed_dict={batch_state: [state],
                                                        batch_stepNo: [ep]})[0]
            #action = np.argmax(SV,1)[0]
        return action
    
    def update_parameters(self,oldstate, newstate, action, reward, done, ep):
        # Get the target values
        if done == False:
            target = reward + 0.99*sess.run(maxQ, 
                     feed_dict = {
                    batch_state : newstate,
                    batch_stepNo: ep
                })
        else:
            target = reward
        
        
         # Update parameters towards the target value
        sess.run(optimizer, 
                 feed_dict = {
                batch_state : oldstate,
                batch_trueQ : target,
                batch_action : action,
                batch_stepNo : ep
            })
        return target
    def get_optimal_policy(self):
        optimal = sess.run(optimal_action, feed_dict = {batch_state : range(env.observation_space.n),
                                                        batch_stepNo : np.repeat(100,env.observation_space.n)
            })
        optimal_name = [ self.inv_action_table[v] for v in optimal]

        return np.reshape(optimal_name,(np.sqrt(env.observation_space.n), np.sqrt(env.observation_space.n)))
    
    def save_trained_policy(self):
        optimal = sess.run(optimal_action, feed_dict = {batch_state : range(env.observation_space.n)
            })
        trained_policy = {}
        for s in range(env.observation_space.n):
            trained_policy[s] = sess.run(optimal_action, feed_dict = {batch_state : [s]})[0]
        self.trained_policy = trained_policy
        
    def epsilon_decay(self):
        self.r_prob *= self.r_prob_decay
        self.r_prob = np.maximum(0.1,self.r_prob) # Never let random probability be less than 5%

In [11]:
### TF-MODEL SPECIFICATION
model = tf_RL_model()

graph = tf.Graph()
with graph.as_default():
    batch_state = tf.placeholder(tf.int64, shape = ([None]), name = 'ph_state')
    batch_stepNo = tf.placeholder(tf.int64, shape = ([None,]), name = 'ph_stepNo')
    batch_action = tf.placeholder(tf.int64, shape = ([None]), name = 'ph_action')
    batch_reward = tf.placeholder(tf.float32, shape = ([None]), name = 'ph_reward')
    batch_trueQ = tf.placeholder(tf.float32, shape = ([None]), name = 'ph_trueQ')

    #W = tf.Variable(tf.constant(value = np.linspace(start = 1, stop = env.observation_space.n*env.action_space.n, 
    #                                                num = env.observation_space.n*env.action_space.n, dtype='float32'), 
    #                shape = [env.observation_space.n, env.action_space.n]
    #               ))
    
    hsize = 100
    # One hot vectors of state and action indicies:
                        
    state = tf.one_hot(indices = batch_state,
                           depth = env.observation_space.n,
                          on_value = 1.0, off_value = 0.0)
    W1 = tf.get_variable(shape = [env.observation_space.n, hsize],
        initializer = tf.random_normal_initializer(mean=0, stddev= 0.1) , name = 'W1')  

    Wstep = tf.get_variable(shape = [200, hsize],
        initializer = tf.random_normal_initializer(mean=0, stddev= 0.1) , name = 'Wstep')
    
    stepNo = tf.one_hot(indices = batch_stepNo,
                           depth = 200,
                          on_value = 1.0, off_value = 0.0)
    timelayer = tf.nn.relu6(tf.matmul(stepNo,Wstep))

    hidden = tf.nn.relu(tf.matmul(state,W1)) + timelayer

    
    W2 = tf.get_variable(shape = [hsize, env.action_space.n],
        initializer = tf.random_normal_initializer(mean=0, stddev= 0.1) , name = 'W2')  
    

    
    
    Qval = tf.matmul(hidden,W2)

    
    action = tf.one_hot(indices = batch_action,
                           depth = env.action_space.n,
                          on_value = 1.0, off_value = 0.0)
    
    optimal_action = tf.argmax(Qval,1)
    
    maxQ = tf.reduce_max(Qval,1)
    
    Q_a = tf.reduce_sum(tf.mul(Qval, action),1) # only get non-zero chosen actions
    loss = tf.nn.l2_loss( batch_trueQ - Q_a)
    optimizer = tf.train.GradientDescentOptimizer(model.learning_rate).minimize(loss)
    saver = tf.train.Saver()

In [ ]:
sess = tf.Session(graph=graph)
with sess as session:
    sess.run(tf.initialize_all_variables())

    # Probability of random action
    #
    model.r_prob_decay = 0.99995
    model.learning_rate = 0.1

    # Logging variables
    visits = np.zeros([env.observation_space.n])

    run_reward = [] # running average initialize
    num_episodes = 50000 # Total number of games
    #old_optimal = model.get_optimal_policy() # Initial optimal strategy
    
    newweight = sess.run(tf.trainable_variables())[0]
    for n in xrange(num_episodes + 1):
        newstate = env.reset()
        done = False

        for ep in xrange(200):
            # Current state
            eprate = ep/200.0
            oldstate = newstate
            # Perform epsilon-greedy action:
            action = model.epsilon_greedy_action(oldstate, eprate)
            # Take action and observe state and reward
            newstate, reward, done, info = env.step(action)
            visits[newstate] += 1

            # Reward modifications:
            # if (done is True and reward == 0):
            #    reward = -0.2
            # reward -= 0.01 # cost of life
            
            
            # Update parameters
            tQ = model.update_parameters([oldstate], [newstate], [action], [reward],done, [eprate]) # brackets fordi det er enkelttall inn
            
            # Debugging:
            if False:
                oldweight = newweight
                newweight = sess.run(tf.trainable_variables())[0]
                diff = newweight - oldweight
                print 'old: ' + str(oldstate) + '\t new: ' + str(newstate) +\
                '\t a: ' + str(action) + '\t r: ' + str(reward) +\
                '\t targetQ: ' + str(np.round(tQ[0],3)) + '\t Qval: ' + str(np.round(oldweight[oldstate,action],3)) + '\t Qdiff: ' + str(np.round(diff[oldstate,action],3)) 

            
            if done:
                break
            
            # print 'episode ' + str(n) + 'reached maximum steps. skipping...'

        # Decay epsilon
        model.epsilon_decay()
        run_reward.append(reward)

        if n % int(num_episodes/10) == 0:
            
            run_avg = np.mean(run_reward)
            run_reward = []
            print 'step: ' + str(n) + ' \t r_prob:' + str(np.round(model.r_prob,2)) + '\t run_avg: ' + str(np.round(run_avg,3))
            print model.get_optimal_policy()
            #print sess.run(tf.trainable_variables())[0]
    #model.save_trained_policy()
    save_path = saver.save(sess, 'tf-frozen8x8.ckpt')

    

step: 0 	 r_prob:0.5	 run_avg: 0.0
[['L' 'R' 'D' 'R' 'D' 'L' 'L' 'L']
 ['L' 'D' 'L' 'L' 'R' 'L' 'R' 'L']
 ['L' 'R' 'L' 'L' 'L' 'L' 'R' 'L']
 ['L' 'L' 'D' 'L' 'L' 'U' 'L' 'L']
 ['L' 'R' 'L' 'D' 'L' 'L' 'L' 'D']
 ['R' 'L' 'L' 'L' 'R' 'L' 'D' 'D']
 ['L' 'L' 'D' 'L' 'L' 'L' 'L' 'D']
 ['D' 'L' 'L' 'L' 'D' 'L' 'L' 'L']]


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Number of visits per episode:
plt.plot(visits/num_episodes)
#num_episodes/visits

### Test Run

In [ ]:
passive_policy = {
    0 : 2,
    1 : 2,
    2 : 2,
    3 : 2,
    4 : 2,
    5 : 2,
    6 : 2,
    7 : 1,
    8 : 3,
    9:3,
    10:3,
    11:3,
    12:3,
    13:3,
    14:3,
    15:2,
    23:2,
    31:2,
    39:2,
    47:2,
    55:2,
    64:2
}

In [ ]:
env.monitor.start('recordings', force=True)
num_episodes = 1000
R = []


def moving_average(a, n = 100) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


model.r_prob = 0 # ensure that only the optimal solution is used

for n in xrange(num_episodes):
    newstate = env.reset()
    done = False
    k = 0
    while not done:
        # Current state
        oldstate = newstate
        
        # Perform epsilon-greedy action:
        action = model.trained_policy[oldstate]
        #action = passive_policy[oldstate]
        # Take action and observe state and reward
        newstate, reward, done, info = env.step(action)
        k += 1
    R.append(reward)
    print 'Episode: ' + str(n) +'\t steps: ' + str(k) + '\t state: ' + str(newstate), '\t reward: ' + str(reward)
    #if n % 1000 == 101:
    #    MA = np.max(moving_average(R))
    #    print 'step: ' + str(n) + '\t MRA: ' + str(MA)
    #    if MA > 0.99:
    #        break
env.monitor.close()

In [ ]:
MV = moving_average(R, n = 100)
print np.max(MV)

plt.plot(MV)

In [ ]:
gym.upload('/notebooks/hjem/RL/recordings', api_key='sk_znZbtlUTlu1nJNqFLRIyA')

In [ ]:
env.monitor.close()
